In [30]:
import numpy as np
import pandas as pd
import plotly.express as px
from sklearn.preprocessing import StandardScaler

In [31]:
# Указываем параметры
Z = 1  # Выбираем гипергеометричское распределение
P = 3  # Число признаков
N = 5  # Число классов

M = 100  # Число наблюдений

In [32]:
data = []
for i in range(N):

    ngood = np.random.randint(70, 100)
    nbad = 100 - ngood
    nsample = np.random.randint(1, 100)

    c = np.random.hypergeometric(ngood=ngood, nbad=nbad, nsample=nsample, size=(M, P))
    # ngood - количество "хороших" элементов в выборке
    # nbad - количество "плохих" элементов в выборке
    # nsample - общий размер выборки
    # size - форма матрицы данных, которую

    data.append(c)

In [33]:
# Собираем признаки из всех классов
xs = np.concatenate([c[:, 0] for c in data])  # Признаки по оси X
ys = np.concatenate([c[:, 1] for c in data])  # Признаки по оси Y
zs = np.concatenate([c[:, 2] for c in data])  # Признаки по оси Z

xsyszs = [[xs[i], ys[i], zs[i]] for i in range(len(xs))]  # Собираем тройки признаков

xsyszs = [[xs[i], ys[i], zs[i]] for i in range(len(xs))]

In [34]:
# Создаем список Y, присваивая каждому классу уникальное значение
# Y будет представлять классификационный признак,
# который указывает на принадлежность каждой точки данных к одному из 5 классов.
Y = []
for i in range(N):
    Y.extend([i] * M)

# Преобразуем список Y в массив numpy
Y = np.array(Y)

In [35]:
# Создаем DataFrame `res_df` из списка `xsyszs`, где `xsyszs` содержит тройки признаков (x, y, z) для каждой точки данных.
res_df = pd.DataFrame(xsyszs)

# Создаем трехмерный рассеянный график с использованием библиотеки Plotly Express.
# Аргументы `x=0`, `y=1`, `z=2` указывают столбцы DataFrame для координат осей X, Y и Z соответственно.
# Аргумент `color=Y` определяет цвет точек на графике в соответствии с классификационным признаком из массива `Y`.
fig = px.scatter_3d(res_df, x=0, y=1, z=2, color=Y)

# Отображаем созданный трехмерный график.
fig.show()


In [36]:
# Стандартизируем данные
sc = StandardScaler()
xsys_n = pd.DataFrame(sc.fit_transform(res_df))
xsys_n.describe()

,0,1,2
count,500.000000,500.000000,5.000000e+02
mean,0.000000,0.000000,-1.705303e-16
std,1.001002,1.001002,1.001002e+00
min,-1.528382,-1.530978,-1.530833e+00
25%,-0.653669,-0.658538,-6.552470e-01
50%,-0.087677,-0.094018,-1.144442e-01
75%,0.787037,0.829742,7.868939e-01
max,1.507389,1.496902,1.507964e+00


In [37]:
# Импортируем класс KMeans из библиотеки scikit-learn для кластеризации данных.
from sklearn.cluster import KMeans

# Создаем объект KMeans с параметрами:
# - n_clusters=5 указывает, что мы хотим разделить данные на 5 кластера.
# - random_state=0 используется для установки начального состояния генератора случайных чисел и обеспечения воспроизводимости результатов.
kmeans = KMeans(n_clusters=5, random_state=0)

# Обучаем модель K-Means на стандартизированных данных `xsys_n`.
kmeans.fit(xsys_n)


/Users/malinatrash/anaconda3/envs/Nerual-Network-Technologies/lib/python3.11/site-packages/sklearn/cluster/_kmeans.py:870: FutureWarning:

The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning



In [38]:
# Создаем DataFrame `df_cluster_centers` для хранения координат центров кластеров.
df_cluster_centers = pd.DataFrame(kmeans.cluster_centers_)

# Выводим DataFrame `df_cluster_centers`, который содержит координаты центров кластеров.
print(df_cluster_centers)

          0         1         2
0  1.417345  1.405039  1.413710
1 -0.614564 -0.616969 -0.609408
2 -1.432678 -1.430391 -1.434003
3 -0.098482 -0.110441 -0.107749
4  0.728379  0.752762  0.737449


In [39]:
# Импортируем функцию confusion_matrix из библиотеки scikit-learn для вычисления матрицы ошибок.
from sklearn.metrics import confusion_matrix

# Выводим матрицу ошибок, которая показывает, как точки данных распределены по кластерам
# и сравнивает это с истинными метками классов.
# `Y` содержит истинные метки классов, а `kmeans.labels_` содержит предсказанные метки кластеров.
confusion_mat = confusion_matrix(Y, kmeans.labels_)
print(confusion_mat)

[[  0   0   0 100   0]
 [100   0   0   0   0]
 [  0   0   0   0 100]
 [  0   0 100   0   0]
 [  0 100   0   0   0]]


In [41]:
import plotly.graph_objects as go
fig = go.Figure(data= [
    go.Scatter3d(
        x=xsys_n[0], y=xsys_n[1], z=xsys_n[2], mode='markers', marker=dict(size=2, color=kmeans.labels_)
    ),
    
    go.Scatter3d(
        x=df_cluster_centers[0],
        y=df_cluster_centers[1],
        z=df_cluster_centers[2],
        mode='markers',
        marker=dict(size=6, color='RED')
    )
])
fig.show()